In [1]:
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
from nltk import ngrams

In [2]:
text=open('speeches.txt','rt',encoding='utf-8')

In [3]:
fin=text.read()

In [4]:
fin=fin.replace('\n', '')
fin=fin.replace('.','. ')

In [5]:
sen=sent_tokenize(fin)
text.close()

In [6]:
final=[]
for i in sen:
    final.append('<s> '+i.replace('.','').replace('?','').replace(',','').replace(' - ','')+' </s>')

In [7]:
xTrain, xTest = train_test_split(final, test_size = 0.2, random_state = 0)

In [9]:
big=dict()
n = 2
bc=0
for sen in xTrain:
    bigrams = ngrams(sen.split(), n)
    for x in bigrams:
        if(x not in big):
            big[x]=1
        else:
            big[x]+=1
        bc=bc+1       

In [10]:
tri=dict()
n = 3
tc=0
for sen in xTrain:
    trigrams = ngrams(sen.split(), n)
    for x in trigrams:
        if(x not in tri):
            tri[x]=1
        else:
            tri[x]=tri[x]+1
        tc=tc+1    

In [11]:
uni=dict()
n = 1
uc=0
for sen in xTrain:
    unigrams = ngrams(sen.split(), n)
    for x in unigrams:
        if(x[-1]=='.'):
            x=x[:-2]
        if(x not in uni):
            uni[x]=1
        else:
            uni[x]=uni[x]+1
        uc=uc+1    


In [12]:
quad=dict()
n = 4
qc=0
for sen in xTrain:
    quadgrams = ngrams(sen.split(), n)
    for x in quadgrams:
        if(x not in quad):
            quad[x]=1
        else:
            quad[x]=quad[x]+1
        qc=qc+1    

In [13]:
uc,bc,tc,qc

(162132, 148583, 135034, 121899)

In [14]:
len(uni),len(big),len(tri),len(quad)

(8043, 50617, 90373, 103141)

In [15]:
MLEB=dict()
for i in big:
    MLEB[i]=big[i]/uni[i[:-1]]

In [16]:
MLET=dict()
for i in tri:
    MLET[i]=tri[i]/big[i[:-1]]

In [17]:
MLEQ=dict()
for i in quad:
    MLEQ[i]=quad[i]/tri[i[:-1]]

In [19]:
MLEU=dict()
for i in uni:
    MLEU[i]=uni[i]/uc

In [20]:
import numpy as np

In [145]:
keys, values = zip(*MLEB.items()) 
values=list(values)
x='<s>'
b=[]
c=[]
print(x)
for i in range(100):
    s=0
    for j in keys:
        if(j[0]==x):
            c.append(MLEB[j])
            b.append(j)
            s=1
    if(s==0):
        break
    array=np.random.multinomial(1 , c , size=1)
    for k in range(len(array[0])):
        if(array[0][k]==1):
            print(str(b[k][1]),end=' ')
            x=b[k][1]
    c=[]
    b=[]     

<s>
And so far behind for everything exactly but they can see what did I SAW CAMERAS OVER THE TRUMP NEVER EVER DONE TWO THINGS -- I just got a disaster which was watching it " </s> 

In [373]:
def predict_first(gram,n):
    x=['<s>']
    b=[]
    c=[]
    for i in range(n-2):
        x=list(x)
        keys, values = zip(*gram[i+1].items()) 
        values=list(values)
        keys=list(keys)
        for j in keys:
            l=list(j)
            if(l[:i+1]==x):
                c.append(gram[i+1][j])
                b.append(j)
#         print(b,c)
        array=np.random.multinomial(1 , c , size=1)
        for k in range(len(array[0])):
            if(array[0][k]==1):
#                 print(str(b[k]),end=' ')
                x=b[k]
        b=[]
        c=[]
    return(list(x))
def generator(gram,n,m):
#     keys, values = zip(*gram[n-2].items()) 
#     values=list(values)
#     x='<s>'
#     b=[]
#     c=[]
#     for j in keys:
#         if(j[0]==x):
#             c.append(gram[n-2][j])
#             b.append(j)
#     c=np.array(c)
#     c=c/sum(c)        
#     array=np.random.multinomial(1 , c , size=1)
#     for k in range(len(array[0])):
#         if(array[0][k]==1):
#             print(str(b[k][:n-1]),end=' ')
#             x=b[k] 
    if(n==1):
        keys, values = zip(*MLEU.items()) 
        values=list(values)
        array=np.random.multinomial(1 , values , size=m)
        print('<s>',end=' ')
        for i in array:
            for j in range(len(keys)):
                if(i[j]==1):
                    if(keys[j][0]=='</s>'):
                        print(str(keys[j][0]),end=' ')
                        return
                    print(str(keys[j][0]),end=' ')
        return
    x=predict_first(gram,n)
    b=[]
    c=[]
    keys, values = zip(*gram[n-1].items()) 
    values=list(values)
    keys=list(keys)
    print(*x,end=' ')
    for i in range(m):
        s=0
        for j in keys:
            l=list(j)
#             print(l[:n-1],x)
            if(l[:n-1]==x):
                c.append(gram[n-1][j])
                b.append(j)
#                 print(j,x,"Match")
                s=1
        if(s==0):
            break
        array=np.random.multinomial(1 , c , size=1)
        for k in range(len(array[0])):
            if(array[0][k]==1):
                print((list(b[k]))[-1],end=' ')
                x=b[k][1:]
        x=list(x)
        c=[]
        b=[]    

In [400]:
MLE=[MLEU,MLEB,MLET,MLEQ]
gram=[uni,big,tri,quad]
n=[uc,bc,tc,qc]
generator(MLE,4,100)


<s> Nobody cares </s> 

In [382]:
def laplace(gram,n,MLE,k):
    for i in range(4):
        for j in gram[i]:
            if(i==0):
                MLE[i][j]=(gram[0][j]+k)/(n[0]+len(gram[0]))
            else:
                MLE[i][j]=(gram[i][j]+k)/(gram[i-1][j[:-1]]+k*(len(gram[0])))

In [410]:
from math import log as log
import math

In [464]:
def perplexity(gram,MLE,xTest,n,k):
    ans=0
    
    for i in xTest:
        ngram = ngrams(sen.split(), n)
        for x in n-gram:
            if(x not in gram[n-1]):
                if(n==1):
                    ans+=log(n[0]+len(gram[0])/k)
                elif(n==2):
                    if()
            else:
                ans+=log(1/gram[n-1][x],2)
    print(math.exp(2,(-1*ans/N))
                

In [476]:
l=perplexity(gram,MLE,xTest,4)

In [485]:
a=[]
for i in xTrain:
    i=i.lower()
    ngram = ngrams(i.split(),3)
    for gram in ngram:
        a.append(list(gram))

[['<s>', 'we', 'can’t'],
 ['we', 'can’t', '</s>'],
 ['<s>', 'unfortunately', 'after'],
 ['unfortunately', 'after', 'the'],
 ['after', 'the', 'cold'],
 ['the', 'cold', 'war'],
 ['cold', 'war', 'our'],
 ['war', 'our', 'foreign'],
 ['our', 'foreign', 'policy'],
 ['foreign', 'policy', 'veered'],
 ['policy', 'veered', 'badly'],
 ['veered', 'badly', 'off'],
 ['badly', 'off', 'course'],
 ['off', 'course', '</s>'],
 ['<s>', 'i', 'in'],
 ['i', 'in', 'a'],
 ['in', 'a', 'lot'],
 ['a', 'lot', 'of'],
 ['lot', 'of', 'money'],
 ['of', 'money', '</s>']]

In [486]:
from keras.preprocessing.text import Tokenizer

In [501]:
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(a)
b=t.texts_to_sequences(a)

In [531]:
b = np.array(b)

In [540]:
x_train=[]
y_train=[]
# for i in b:
#     x_train.append(i[:-1])
#     y_train.append(i[-1])
x_train = b[:,:-1]
y_train = b[:,-1]

In [533]:
from keras.layers import SimpleRNN
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [534]:
model = Sequential()
model.add(Embedding(len(t.word_counts), 50, input_length=2))
# if dropout > 0:
#     model.add(Dropout(dropout))
# model.add(Dense(len(words)))
# model.add(Activation('softmax'))
model.add(SimpleRNN(50, return_sequences=True))
model.add(SimpleRNN(50))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(t.word_counts), activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 2, 50)             326400    
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 2, 50)             5050      
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 50)                5050      
_________________________________________________________________
dense_10 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_11 (Dense)             (None, 6528)              659328    
Total params: 1,000,928
Trainable params: 1,000,928
Non-trainable params: 0
_________________________________________________________________


In [542]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history_rnn = model_rnn.fit(x_train, y_train, batch_size=128, epochs=10)

ValueError: Error when checking target: expected dense_13 to have shape (6528,) but got array with shape (1,)

In [536]:
y_train= to_categorical(y_train, num_classes=len(t.word_counts))

MemoryError: 

In [545]:
# VANILLA RNN MODEL
from keras.layers import SimpleRNN
vocab_size=len(t.word_counts)
model_rnn = Sequential()
model_rnn.add(Embedding(vocab_size, 50, input_length=2))
model_rnn.add(SimpleRNN(25, return_sequences=True))
model_rnn.add(SimpleRNN(25))
model_rnn.add(Dense(50, activation='relu'))
model_rnn.add(Dense(vocab_size, activation='softmax'))

model_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 2, 50)             326400    
_________________________________________________________________
simple_rnn_11 (SimpleRNN)    (None, 2, 25)             1900      
_________________________________________________________________
simple_rnn_12 (SimpleRNN)    (None, 25)                1275      
_________________________________________________________________
dense_14 (Dense)             (None, 50)                1300      
_________________________________________________________________
dense_15 (Dense)             (None, 6528)              332928    
Total params: 663,803
Trainable params: 663,803
Non-trainable params: 0
_________________________________________________________________


In [550]:
model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history_rnn = model_rnn.fit(x_train, y_train, batch_size=128, epochs=10)

ValueError: Error when checking target: expected dense_15 to have shape (6528,) but got array with shape (1,)

In [549]:
np.shape(x_train),np.shape(y_train)

((135034, 2), (135034,))

In [552]:
# NEURAL APPROACH
sequences = list()
n_gram = 3


# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(a)
sequences_intEncoded = tokenizer.texts_to_sequences(a)

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

sequences_intEncoded[:10]

sequences_intEncoded = np.array(sequences_intEncoded)
np.shape(sequences)

X = sequences_intEncoded[:,:-1]
y = sequences_intEncoded[:,-1]

y = to_categorical(y, num_classes=vocab_size)

# VANILLA RNN MODEL
from keras.layers import SimpleRNN

model_rnn = Sequential()
model_rnn.add(Embedding(vocab_size, 50, input_length=n_gram-1))
model_rnn.add(SimpleRNN(25, return_sequences=True))
model_rnn.add(SimpleRNN(25))
model_rnn.add(Dense(50, activation='relu'))
model_rnn.add(Dense(vocab_size, activation='softmax'))

model_rnn.summary()

model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history_rnn = model_rnn.fit(X, y, batch_size=128, epochs=10)

6529
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 2, 50)             326450    
_________________________________________________________________
simple_rnn_13 (SimpleRNN)    (None, 2, 25)             1900      
_________________________________________________________________
simple_rnn_14 (SimpleRNN)    (None, 25)                1275      
_________________________________________________________________
dense_16 (Dense)             (None, 50)                1300      
_________________________________________________________________
dense_17 (Dense)             (None, 6529)              332979    
Total params: 663,904
Trainable params: 663,904
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
135034/135034 [==============================] - 46s 338us/step - loss: 5.8593 - acc: 0.1244
Epoch 2/10
135034/135034 [=